# **Стандарден Проект: Тестирање и истражување на адаптивни тренинг планови**

Овој код преставува целосен процес за истражување и развој на систем за
адаптивни тренинг планови. Користи податочно-воден пристап,
каде симулациите се базираат на реални и случајно избрани записи од датасетот calories.csv, најден на kaggle преку линкот https://www.kaggle.com/datasets/ruchikakumbhar/calories-burnt-prediction.


# **Секција 1: Подготовка на работна околина и внесување на соодветни библиотеки**

In [29]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import random
import warnings
warnings.filterwarnings('ignore')
print("Библиотеките се успешно внесени.")

Библиотеките се успешно внесени.


# **Секција 2: Вчитување на податочното множество calories.csv и сите негови категории**

In [30]:
try:
    !gdown 1IkTe0p2Tnh-ViTIdBiGZclRwnGw23z8X
    df = pd.read_csv('calories.csv')
    print("Податочното множество 'calories.csv' е успешно вчитано.")
except FileNotFoundError:
    print("ГРЕШКА: Податочното множество 'calories.csv' не е пронајдено.")
    exit()

Downloading...
From: https://drive.google.com/uc?id=1IkTe0p2Tnh-ViTIdBiGZclRwnGw23z8X
To: /content/calories.csv
100% 742k/742k [00:00<00:00, 16.2MB/s]
Податочното множество 'calories.csv' е успешно вчитано.


# **Секција 3: Feature engineering и воведување колона за класификација на типови на адаптивни тренинзи**

In [31]:
def definiraj_plan(row):
    duration = row['Duration']; heart_rate = row['Heart_Rate']
    if duration <= 10:
        if heart_rate < 90: return "Кратка релаксација"
        elif heart_rate < 115: return "Кратка кардио активност"
        else: return "Краток висок интензитет (HIIT)"
    elif duration <= 20:
        if heart_rate < 95: return "Лесен тренинг за издржливост"
        elif heart_rate < 115: return "Стандарден кардио тренинг"
        else: return "Интензивен кардио тренинг"
    else:
        if heart_rate < 100: return "Долг тренинг со низок интензитет"
        elif heart_rate < 115: return "Тренинг за издржливост"
        else: return "Напреден тренинг за издржливост"
df['Training_Plan_Type'] = df.apply(definiraj_plan, axis=1)
print("Успешно е креирана нова колона 'Training_Plan_Type'.")

Успешно е креирана нова колона 'Training_Plan_Type'.


# **Секција 4: Градење на моделот за предвидување**

In [32]:
df_model = df.copy(); df_model['Gender'] = df_model['Gender'].map({'male': 0, 'female': 1})
features = ['Gender', 'Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp']
target = 'Calories'
X = df_model[features]; y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
accuracy_r2 = model.score(X_test, y_test)
print(f"Моделот за предвидување е валидиран со точност (R-squared) од {accuracy_r2*100:.2f}%.")

Моделот за предвидување е валидиран со точност (R-squared) од 99.89%.


# **Секција 5.1: Дефинирање на функции за практика - алтернативни планови**

In [33]:
def pronajdi_alternativni_planovi(cel_kalorii, korisnik_profil, tolerancija=15):
    print(f"\n СЦЕНАРИО 1: Пребарување на АЛТЕРНАТИВНИ планови за цел од ~{cel_kalorii:.0f} калории.")
    najdeni_planovi = []
    prikazani_kalorii = set() # Сет за следење на веќе прикажани вредности
    for duration in range(10, 61, 2):
        for heart_rate in np.arange(90, 141, 2.5):
            body_temp_est = 38.0 + (heart_rate - 90) * 0.05
            podatoci_za_predviduvanje = {**korisnik_profil, 'Duration': duration, 'Heart_Rate': heart_rate, 'Body_Temp': body_temp_est}
            input_df = pd.DataFrame([podatoci_za_predviduvanje], columns=X.columns)
            predvideni_kalorii = round(float(model.predict(input_df)[0]), 1)

            # Проверуваме дали предвидувањето е во рамки на целта
            if abs(predvideni_kalorii - cel_kalorii) <= tolerancija:
                # После корекција прикажуваме само опции со различна проценета потрошувачка на калории
                if predvideni_kalorii not in prikazani_kalorii:
                    plan = {'Duration': duration, 'Heart_Rate': int(heart_rate), 'Predicted_Calories': predvideni_kalorii}
                    plan['Plan_Type'] = definiraj_plan(plan)
                    najdeni_planovi.append(plan)
                    prikazani_kalorii.add(predvideni_kalorii)
                if len(najdeni_planovi) >= 3: return najdeni_planovi
    return najdeni_planovi


# **Секција 5.2: Дефинирање на функции за практика - адаптивен тренинг во реално време**

In [34]:
def adaptiraj_trening_vo_realno_vreme(planiran_trening, momentalen_status, korisnik_profil):
    # Оваа функција останува иста во логиката, но ќе биде повикана со пореални податоци
    print(f"\n🔄 СЦЕНАРИО 2: Адаптација на тренинг во реално време.")
    print(f"   План: {planiran_trening['Duration']} минути за цел од ~{planiran_trening['Target_Calories']:.0f} калории.")
    preostanato_vreme = planiran_trening['Duration'] - momentalen_status['Vreme_izminato']
    if preostanato_vreme <= 0: return
    proekcija_temp = 38.0 + (momentalen_status['Prosechen_puls_dosega'] - 90) * 0.05
    podatoci_za_proekcija = {**korisnik_profil, 'Duration': preostanato_vreme, 'Heart_Rate': momentalen_status['Prosechen_puls_dosega'], 'Body_Temp': proekcija_temp}
    input_df_proekcija = pd.DataFrame([podatoci_za_proekcija], columns=X.columns)
    proektirani_kalorii_za_ostatok = model.predict(input_df_proekcija)[0]
    vkupno_proektirani_kalorii = momentalen_status['Potrosheni_dosega'] + proektirani_kalorii_za_ostatok
    print(f"   Статус: По {momentalen_status['Vreme_izminato']:.0f} мин, потрошени се {momentalen_status['Potrosheni_dosega']:.0f} кал. со просечен пулс од {momentalen_status['Prosechen_puls_dosega']:.0f} bpm.")
    print(f"   Проекција: Ако се продолжи со ова темпо, вкупната потрошувачка ќе биде ~{vkupno_proektirani_kalorii:.0f} калории.")

    # После корекција системот може да прикаже и позитивна и негативна препорака
    if vkupno_proektirani_kalorii < planiran_trening['Target_Calories'] * 0.95: # Ако заостанува за повеќе од 5%
        potrebni_kalorii_za_ostatok = planiran_trening['Target_Calories'] - momentalen_status['Potrosheni_dosega']
        potreben_puls = momentalen_status['Prosechen_puls_dosega']
        for puls in range(int(potreben_puls) + 1, 155):
            korigirana_temp = 38.0 + (puls - 90) * 0.05
            podatoci_za_korekcija = {**korisnik_profil, 'Duration': preostanato_vreme, 'Heart_Rate': puls, 'Body_Temp': korigirana_temp}
            input_df_korekcija = pd.DataFrame([podatoci_za_korekcija], columns=X.columns)
            if model.predict(input_df_korekcija)[0] >= potrebni_kalorii_za_ostatok:
                potreben_puls = puls
                break
        print(f"   ПРЕДУПРЕДУВАЊЕ: Моменталното темпо е недоволно за да се постигне целта.")
        print(f"   АДАПТИВНА ПРЕПОРАКА: За да ја постигнете целта од {planiran_trening['Target_Calories']:.0f} калории, обидете се да го зголемите пулсот на ~{potreben_puls} bpm за преостанатите {preostanato_vreme:.0f} минути.")
    elif vkupno_proektirani_kalorii > planiran_trening['Target_Calories'] * 1.1: # Ако надминува за повеќе од 10%
        print(f"   ОДЛИЧНО! Вежбате со посилен интензитет од планираното. Можете малку да го намалите темпото ако сакате.")
    else:
        print(f"   ОДЛИЧНО! Перформансите се во согласност со планот. Само продолжете така.")

# **Секција 6: Податочно-водена динамичка симулација**

In [35]:
    # Избор на случаен, но реален запис од датасетот за основа на симулацијата
    random_index = random.randint(0, len(df) - 1)
    real_data_point = df.loc[random_index]

    # Креирање на кориснички профил и цел од реалниот запис
    korisnik_profil = {
        'Gender': 0 if real_data_point['Gender'] == 'male' else 1,
        'Age': real_data_point['Age'],
        'Height': real_data_point['Height'],
        'Weight': real_data_point['Weight'],
    }
    cel_kalorii = real_data_point['Calories']

    gender_text = "Женско" if korisnik_profil['Gender'] == 1 else "Машко"
    print(f"\nСимулацијата се базира на реален запис од датасетот:")
    print(f"   > Корисник: {gender_text}, {korisnik_profil['Age']} год, {korisnik_profil['Height']} cm, {korisnik_profil['Weight']} kg.")
    print(f"   > Неговиот/Нејзиниот оригинален тренинг потрошил {cel_kalorii:.0f} калории.")

    # Демонстрација на Сценарио 1: Наоѓање алтернативи на реален тренинг
    alternativni_planovi = pronajdi_alternativni_planovi(cel_kalorii=cel_kalorii, korisnik_profil=korisnik_profil)

    if alternativni_planovi:
        print("\n   РЕЗУЛТАТ: Пронајдени се следниве алтернативни тренинзи:\n")
        for i, plan in enumerate(alternativni_planovi):
            print(f"   Алтернатива {i+1}: '{plan['Plan_Type']}'")
            print(f"                   (Времетраење: {plan['Duration']} мин, Прос. пулс: {plan['Heart_Rate']} bpm) -> Проценети {plan['Predicted_Calories']} калории.")

        # Демонстрација на Сценарио 2: Адаптација на реален тренинг
        planiran_trening = {'Duration': real_data_point['Duration'], 'Target_Calories': cel_kalorii}

        # Симулираме отстапување од реалниот тренинг (или подобро или полошо)
        vreme_izminato = planiran_trening['Duration'] / 2
        scenario_type = random.choice(['underperforming', 'on_track', 'overperforming'])

        if scenario_type == 'underperforming':
            # Корисникот потрошил помалку калории и имал понизок пулс
            potroseni_dosega = (cel_kalorii / 2) * random.uniform(0.7, 0.9)
            prosechen_puls_dosega = real_data_point['Heart_Rate'] * random.uniform(0.85, 0.95)
        else:
            # Корисникот потрошил повеќе калории и имал повисок пулс
            potroseni_dosega = (cel_kalorii / 2) * random.uniform(1.0, 1.2)
            prosechen_puls_dosega = real_data_point['Heart_Rate'] * random.uniform(1.0, 1.1)

        status_na_korisnikot = {'Vreme_izminato': vreme_izminato, 'Potrosheni_dosega': potroseni_dosega, 'Prosechen_puls_dosega': prosechen_puls_dosega}

        adaptiraj_trening_vo_realno_vreme(
            planiran_trening=planiran_trening,
            momentalen_status=status_na_korisnikot,
            korisnik_profil=korisnik_profil
        )
    else:
        print(f"\n   РЕЗУЛТАТ: Не се пронајдени доволно различни алтернативни планови за цел од {cel_kalorii:.0f} калории.")


Симулацијата се базира на реален запис од датасетот:
   > Корисник: Машко, 23 год, 197.0 cm, 97.0 kg.
   > Неговиот/Нејзиниот оригинален тренинг потрошил 181 калории.

 СЦЕНАРИО 1: Пребарување на АЛТЕРНАТИВНИ планови за цел од ~181 калории.

   РЕЗУЛТАТ: Пронајдени се следниве алтернативни тренинзи:

   Алтернатива 1: 'Напреден тренинг за издржливост'
                   (Времетраење: 22 мин, Прос. пулс: 120 bpm) -> Проценети 171.1 калории.
   Алтернатива 2: 'Напреден тренинг за издржливост'
                   (Времетраење: 22 мин, Прос. пулс: 122 bpm) -> Проценети 179.5 калории.
   Алтернатива 3: 'Напреден тренинг за издржливост'
                   (Времетраење: 22 мин, Прос. пулс: 125 bpm) -> Проценети 186.5 калории.

🔄 СЦЕНАРИО 2: Адаптација на тренинг во реално време.
   План: 30.0 минути за цел од ~181 калории.
   Статус: По 15 мин, потрошени се 100 кал. со просечен пулс од 106 bpm.
   Проекција: Ако се продолжи со ова темпо, вкупната потрошувачка ќе биде ~185 калории.
   ОДЛИЧНО!